In [ ]:
import numpy as np
import torch
import cv2
import os
from utils import *
import numpy as np
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

In [ ]:
## load the data
path = os.getcwd() + '/../data/'  
images_np = images_np_load(path, plot=False)

In [ ]:
def get_padded_image(image, pad):
    """给图像周围补零，方便边界处理"""
    if image.ndim == 3:
        return np.pad(image, ((pad,pad),(pad,pad),(0,0)), mode='constant')
    else:
        return np.pad(image, ((pad,pad),(pad,pad)), mode='constant')


def log_polar_offsets(regionsize, num_r=4, num_theta=20):
    """生成 log-polar 相对坐标偏移表"""
    r = np.logspace(0, np.log10(regionsize / 2), num_r) - 1
    #r = np.linsapce(0, r_max, num_r)
    theta = np.linspace(0, 2*np.pi, num_theta, endpoint=False)
    
    rr, tt = np.meshgrid(r, theta, indexing='ij')
    dy = (rr * np.sin(tt)).astype(int)
    dx = (rr * np.cos(tt)).astype(int)
    offsets = np.stack([dy, dx], axis=-1)  # shape (num_r,num_theta,2)

    return offsets


def self_similarity_logpolar_fast(image, regionsize=81, patchsize=5, alpha=0.1, num_r=4, num_theta=20):
    H, W = image.shape[:2]
    out_H, out_W = H - regionsize + 1, W - regionsize + 1
    pad = regionsize // 2 + patchsize // 2

    padded = get_padded_image(image, pad)
    offsets = log_polar_offsets(regionsize, num_r, num_theta)
    sim_map = np.zeros((out_H, out_W, num_r, num_theta), dtype=np.float32)

    # 枚举所有中心点
    y_coords, x_coords = np.arange(out_H) + pad, np.arange(out_W) + pad

    for i, y in enumerate(y_coords):
        for j, x in enumerate(x_coords):
            # 中心 patch
            yc, xc = y, x
            cp = padded[yc - patchsize//2 : yc + patchsize//2 + 1,
                        xc - patchsize//2 : xc + patchsize//2 + 1]
            # log-polar 索引
            ys = yc + offsets[:,:,0]
            xs = xc + offsets[:,:,1]


            for r_idx in range(num_r):
                for t_idx in range(num_theta):
                    y_patch = ys[r_idx, t_idx]
                    x_patch = xs[r_idx, t_idx]
                    patch = padded[y_patch : y_patch+patchsize,
                                   x_patch : x_patch+patchsize]
                    sim_map[i,j,r_idx,t_idx] = np.exp(-alpha * np.sum((cp - patch)**2))
    return sim_map


# 示例调用
image = np.transpose(images_np[0], [1,2,0])
sim_map_fast = self_similarity_logpolar_fast(image)
print(sim_map_fast.shape)  # (H', W', num_r, num_theta)


In [ ]:
def visualize_logpolar_feature(sim_feature):
    """
    sim_feature: shape (num_r, num_theta)
    """
    num_r, num_theta = sim_feature.shape
    
    # 扩展半径数组和角度数组
    r = np.arange(1, num_r+1)  # 可以换成实际 log 半径
    theta = np.linspace(0, 2*np.pi, num_theta, endpoint=False)
    
    # meshgrid，用于极坐标绘图
    Theta, R = np.meshgrid(theta, r)
    
    # 创建极坐标图
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    
    # 绘制扇形图
    c = ax.pcolormesh(Theta, R, sim_feature, shading='auto', cmap='viridis')
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    fig.colorbar(c, ax=ax, label='Similarity')
    plt.show()


In [ ]:

visualize_logpolar_feature(sim_map_fast[0][0])


In [ ]:
def visualize_logpolar_feature(sim_feature, r):
    _ , num_theta = sim_feature.shape 
    theta = np.linspace(0, 2*np.pi, num_theta, endpoint=False)
    Theta, R = np.meshgrid(theta, r, indexing='xy')
    # 创建极坐标图
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    # 绘制扇形图
    c = ax.pcolormesh(Theta, R, sim_feature, shading='auto', cmap='gray')
    ax.set_theta_zero_location("E")  
    ax.set_theta_direction(1)        
    fig.colorbar(c, ax=ax, label='Similarity')
    plt.show()

def visualize_similarity_map(similarity_map):
    # 使用imshow显示灰度图像
    plt.imshow(similarity_map, cmap='gray')  # cmap='gray'表示以灰度显示
    plt.colorbar()  # 显示颜色条，帮助了解灰度值的范围
    plt.title("Similarity Map")  # 可选：为图像添加标题
    plt.axis('off')  # 可选：关闭坐标轴显示
    plt.show()

def gaussian_kernel(size: int, sigma: float):
    kernel = torch.zeros((size, size), dtype=torch.float32)
    center = size // 2
    for i in range(size):
        for j in range(size):
            dist = (torch.tensor(i, dtype=torch.float32) - center) ** 2 + (torch.tensor(j, dtype=torch.float32) - center) ** 2
            kernel[i][j] = torch.exp(-dist / (2 * sigma ** 2))
    
    kernel /= kernel.sum()
    return kernel


def log_polar_conversion(similarity_map, num_r=4, num_theta=20):
    R = similarity_map.shape[0] // 2
    r = np.linspace(1, R, num_r + 1)
    theta = np.linspace(-2*np.pi / (num_theta * 2), 2*np.pi - 2*np.pi / (num_theta * 2), num_theta + 1)
    
    return r[1:], theta[1:]

def update_log_polar_map(similarity_map, r, theta, num_r=4, num_theta=20):
    H, W = similarity_map.shape
    log_polar_map = np.zeros((num_r, num_theta)) 
    center_y, center_x = H // 2, W // 2  
    
    for i in range(H):
        for j in range(W):
            
            dy, dx = center_y - i, j - center_x
            r_val, t_val = np.sqrt(dy**2 + dx**2), (np.arctan2(dy, dx) + 2*np.pi) % (2*np.pi)
            if t_val > r[-1]:
                t_val = t_val - 2*np.pi
    
            r_idx, t_idx = np.digitize(r_val, r), np.digitize(t_val, theta) % num_theta  
            if r_idx < num_r:
                log_polar_map[r_idx, t_idx] = max(log_polar_map[r_idx, t_idx], similarity_map[i, j])
    
    return log_polar_map